# Homework 5 - Visit the Wikipedia hyperlinks graph!
In this assignment we perform an analysis of the Wikipedia Hyperlink graph. In particular, given extra information about the categories to which an article belongs to, we are curious to rank the articles according to some criteria. 

<div style="text-align:center"><img src="https://i.pinimg.com/originals/a7/5f/dc/a75fdcab110ae11f155ed96f428a86ae.png"/> </div>

## Research questions


**[RQ1]** Build the graph <img src="https://latex.codecogs.com/gif.latex?G=(V,&space;E)" title="G=(V, E)" /> where *V* is the set of articles and *E* the hyperlinks among them, and provide its basic information:
 
- If it is direct or not
- The number of nodes
- The number of edges 
- The average node degree. Is the graph dense?

###### Build the graph!

In [7]:
from collections import defaultdict
import networkx as nx

In [3]:
file = open('wiki-topcats-reduced.txt','r').read().split('\n')
grafo = defaultdict(set)
for row in file:
        link=row.split('\t')
        try:
            grafo[link[0]].add(link[1])
            if link[1] not in grafo:
                grafo[link[1]] = set()
        except: 
            pass

###### Find out if it's directed or not:

We want to check if all the nodes that have edges coming form the node __62__ have an edge to the node __62__.

In [4]:
print(all(["62" in grafo[edge] for edge in grafo['62']]))

False


As you can see, the statement above tells us that not all the nodes that are pointed by the node __62__ have an edge to the node __62__ and this is the counterexample to proof that our graph is directed.

###### Get the number of nodes!

In [5]:
number_of_nodes=len(grafo)
number_of_nodes

461194

###### Get the number of edges!

In [11]:
number_of_edges= sum([len(grafo[node]) for node in grafo])
number_of_edges

2645247

###### Get the average node degree. Is the graph dense?

In graph theory, the degree (or valency) of a vertex of a graph is the number of edges incident to the vertex. The degree of a vertex $v$ is denoted $\deg(v)$.

In [35]:
avg_degree= 2*number_of_edges/number_of_nodes
avg_degree

12.33333255625829

As we see, the average node degree is slightly great than six.
In mathematics, a dense graph is a graph in which the number of edges is close to the maximal number of edges.
We can conclude that the graph is not dense. It is very sparse indeed.

## RQ2 
Given a category $C_0 = \{article_1, article_2, ... \}$ as input we want to rank all of the nodes in V according to the following criteria:

In [12]:
categories = defaultdict(list)
with open('wiki-topcats-categories.txt', 'r') as f:
    for row in f:
        splitted_row = row.split(' ')
        categories[splitted_row[0][9:-1]] = splitted_row[1:]

In [75]:
G = nx.read_edgelist('wiki-topcats-reduced.txt', nodetype=str, delimiter='\t', create_using=nx.DiGraph())

In [58]:
Gprime = G.subgraph(categories['Living_people'])

In [76]:
score={}
for edge in Gprime.edges():
    if edge[1] not in score:
        score[edge[1]]=1
    else:
        score[edge[1]]+=1

In [77]:
score['107']

13

In [78]:
score

{'401135': 55,
 '1069112': 52,
 '1163551': 363,
 '60219': 814,
 '167532': 1,
 '400980': 8,
 '401018': 3,
 '401019': 6,
 '401053': 21,
 '401067': 16,
 '401137': 57,
 '401154': 139,
 '401171': 135,
 '401184': 129,
 '401227': 90,
 '401231': 50,
 '401295': 11,
 '401310': 20,
 '401315': 11,
 '401457': 3,
 '401474': 3,
 '401505': 3,
 '401609': 4,
 '401628': 5,
 '401975': 140,
 '401981': 51,
 '402265': 7,
 '402300': 4,
 '402715': 3,
 '402718': 6,
 '447882': 8,
 '595633': 1,
 '606279': 218,
 '630946': 18,
 '723911': 1,
 '724192': 6,
 '776478': 4,
 '809904': 1,
 '810461': 6,
 '824998': 25,
 '827334': 14,
 '946986': 144,
 '961942': 1,
 '1058269': 43,
 '1060341': 101,
 '1061728': 197,
 '1061824': 236,
 '1061885': 383,
 '1061905': 289,
 '1062323': 60,
 '1062938': 100,
 '1161659': 26,
 '1163338': 118,
 '1163407': 256,
 '1163806': 131,
 '1169888': 14,
 '1184217': 451,
 '1184538': 5918,
 '1245651': 11,
 '1288076': 5,
 '1288276': 4,
 '1394526': 3475,
 '1399606': 3,
 '1571179': 10,
 '1060396': 66,
 '10